# Redes Neuronales Convolucionales

En este _notebook_ vamos a ver cómo construir una red convolucional con Keras. El _dataset_ que vamos a usar es el de [gatos vs perros de Kaggle](https://www.kaggle.com/chetankv/dogs-cats-images). Este tutorial está inspirado en [este post](https://medium.com/@apoorvgupta00/binary-classification-cats-vs-dogs-tutorial-using-tensorflow2-and-keras-bbe07f723d75).

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import math

data_dir = 'dataset' 
os.listdir(data_dir)  
test_p = data_dir + '/test_set/'
train_p = data_dir + '/training_set/'

Aquí vamos a ver las dimensiones de las imágenes para normalizar los tamaños.

In [2]:
dimension1 = []
dimension2 = []

for i_file in os.listdir(test_p+'cats'):
	img= imread(test_p + 'cats/' + i_file)
	x, y, colors = img.shape
	dimension1.append(x)
	dimension2.append(y)
    
print(np.mean(dimension1))
print(np.mean(dimension2)) 
image_shape = (math.ceil(np.mean(dimension1)), math.ceil(np.mean(dimension2)), 3)
image_shape

356.267
413.064


(357, 414, 3)

Aquí vamos a hacer un truco común. Vamos a modificar ligeramente los datos que tenemos con la clase `ImageDataGenerator`. La idea es ir rotando las imágenes y cambiar su alto y ancho, para así generar imágenes a partir de las existentes. Vamos a ver más adelante que deduce el nombre de las clases de los subdirectorios del _dataset_ de entrenamiento y prueba.

In [3]:
image_gen = ImageDataGenerator(rotation_range=10, 
                               width_shift_range=0.10, 
                               height_shift_range=0.10, 
                               rescale=1/255)

# Rescale nos sirve para pasar de valores del 0 al 255 a valores entre 0 y 1. 

print(image_gen.flow_from_directory(train_p))
print(image_gen.flow_from_directory(test_p))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Ahora vamos a instanciar el modelo secuencial.

In [4]:
keras.backend.clear_session()

model = Sequential(
    [
        Conv2D(filters=32, kernel_size=(3,3), input_shape=image_shape, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(64),
        Activation('relu'),
        Dropout(0.5),
        Dense(1), # Queremos una neurona final con una probabilidad, dado que es clasificación binaria.
        Activation('sigmoid')
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 355, 412, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 177, 206, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 175, 204, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 87, 102, 32)       0         
_________________________________________________________________
flatten (Flatten)            (None, 283968)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                18174016  
_________________________________________________________________
activation (Activation)      (None, 64)                0

Como vemos, aquí tenemos hartas capas. Las capas `Conv2D` son las capas convolucionales. La primera capa tiene 32 filtros de $3\times3$. El tamaño del input es `input_shape` y la activación es ReLU. Luego agregamos una capa de pooling de tamaño 2x2. En este caso no estamos añadiendo un tamaño del _stride_ en la capa de _pooling_ y por defecto será del mismo tamaño que `pool_size`. La última capa nueva es `Dropout`. Esta capa desactiva (les asigna valor 0) de forma aleatoria la mitad (0.5 = 50%) de las neuronas de la última capa en la fase de entrenamiento. Esto se usa para preveenir el _overfitting_. Ahora compilaremos el modelo.

In [5]:
# El optimizador ADAM es un (a grandes rasgos) SGD optimizado
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Y ahora lo vamos a entrenar.

In [6]:
batch_size = 32
# target_size me indica el tamaño al que redimensionamos la imagen
train_image_gen = image_gen.flow_from_directory(train_p, 
                                                target_size=image_shape[:2], 
                                                color_mode='rgb', 
                                                batch_size=batch_size, 
                                                class_mode='binary')
test_image_gen = image_gen.flow_from_directory(test_p, 
                                               target_size=image_shape[:2], 
                                               color_mode='rgb', 
                                               batch_size=batch_size,
                                               class_mode='binary',
                                               shuffle=False)
train_image_gen.class_indices

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


{'cats': 0, 'dogs': 1}

In [7]:
model.fit(train_image_gen, epochs=10)

Epoch 1/10
250/250 [==============================] - 699s 3s/step - loss: 0.7049 - accuracy: 0.5775
Epoch 2/10
250/250 [==============================] - 626s 3s/step - loss: 0.6509 - accuracy: 0.6239
Epoch 3/10
250/250 [==============================] - 624s 2s/step - loss: 0.6369 - accuracy: 0.6351
Epoch 4/10
250/250 [==============================] - 669s 3s/step - loss: 0.6069 - accuracy: 0.6701
Epoch 5/10
250/250 [==============================] - 828s 3s/step - loss: 0.5784 - accuracy: 0.7025
Epoch 6/10
250/250 [==============================] - 685s 3s/step - loss: 0.5624 - accuracy: 0.7131
Epoch 7/10
250/250 [==============================] - 748s 3s/step - loss: 0.5482 - accuracy: 0.7245
Epoch 8/10
250/250 [==============================] - 658s 3s/step - loss: 0.5231 - accuracy: 0.7481
Epoch 9/10
250/250 [==============================] - 626s 3s/step - loss: 0.5204 - accuracy: 0.7430
Epoch 10/10
250/250 [==============================] - 623s 2s/step - loss: 0.5157 - accura

Y ahora veremos cómo funciona la predicción.

In [8]:
from sklearn.metrics import classification_report,confusion_matrix

pred_probabilities = model.predict(test_image_gen)
predictions = pred_probabilities > 0.5
print(classification_report(test_image_gen.classes, predictions))
print(confusion_matrix(test_image_gen.classes, predictions))

              precision    recall  f1-score   support

           0       0.73      0.78      0.75      1000
           1       0.76      0.71      0.74      1000

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000

[[782 218]
 [291 709]]
